In [5]:
import requests as req
import pandas as pd
from time import sleep

payload = {'download':'','hdn_gostartdate':'2017/09/1','hdn_goenddate':'2017/11/30','syear':'2017','smonth':'09','sday':'1','eyear':'2017','emonth':'11','eday':'30','datestart':'2017/09/1','dateend':'2017/11/30'}

err_count = 0
while err_count <3:
    try:
        html = req.post('http://www.taifex.com.tw/chinese/3/3_5.asp', data=payload)
        break
    except:
        sleep(5)
        err_count += 1
        continue
if err_count == 3:
    print('連線失敗')

#注意網頁編碼
html.encoding = 'utf-8'
df = pd.read_html(html.text)
df = df[2] #這裡的數字必須從0開始試，直到選取所需資料為止
df.columns = df.iloc[0] #指定第一列為欄位名稱
df = df.drop(0).reset_index(drop=True) #刪除第一列
df.sort_index(ascending=False).head() #以index排序(由新到舊)

,日期,美元／新台幣,人民幣／新台幣,歐元／美元,美元／日幣,英鎊／美元,澳幣／美元,美元／港幣,美元／人民幣,美元／南非幣,紐幣／美元
62,2017/11/30,30.01,4.536972,1.1859,112.295,1.34635,0.75735,7.8082,6.61455,13.637,0.6841
61,2017/11/29,29.99,4.544505,1.1868,111.465,1.3419,0.75895,7.805,6.5992,13.6086,0.69135
60,2017/11/28,30.001,4.544851,1.1904,111.3,1.33285,0.76,7.8033,6.6011,13.7511,0.6925
59,2017/11/27,30.006,4.548369,1.19385,111.24,1.3334,0.76185,7.802,6.5971,14.0394,0.6881
58,2017/11/24,30.01,4.544654,1.18445,111.42,1.33135,0.76115,7.80785,6.60335,13.94585,0.68705


In [6]:
df.to_csv('../data/fp_demo2.csv', sep=',', encoding='utf-8-sig', index=False)

In [7]:
import pandas.io.sql as pd_sql
import sqlite3 as sql

#連線至sqlite檔案，若無該檔案sql，則會建立一個新的
conn = sql.connect("../data/twse.db")
#將Dataframe資料寫入sql檔中的'demo2'表中，無該資料表則會自動建立
df.to_sql("demo2", conn, if_exists="replace", index=False)

In [8]:
pd.read_sql_query("select * from demo2;", conn).sort_index(ascending=False).head() #以index排序(由新到舊)

,日期,美元／新台幣,人民幣／新台幣,歐元／美元,美元／日幣,英鎊／美元,澳幣／美元,美元／港幣,美元／人民幣,美元／南非幣,紐幣／美元
62,2017/11/30,30.01,4.536972,1.1859,112.295,1.34635,0.75735,7.8082,6.61455,13.637,0.6841
61,2017/11/29,29.99,4.544505,1.1868,111.465,1.3419,0.75895,7.805,6.5992,13.6086,0.69135
60,2017/11/28,30.001,4.544851,1.1904,111.3,1.33285,0.76,7.8033,6.6011,13.7511,0.6925
59,2017/11/27,30.006,4.548369,1.19385,111.24,1.3334,0.76185,7.802,6.5971,14.0394,0.6881
58,2017/11/24,30.01,4.544654,1.18445,111.42,1.33135,0.76115,7.80785,6.60335,13.94585,0.68705
